In [ ]:
#Nota la base tenia mas de 800 000 registros y pesaba casi 700 megas por lo que solo pude ocupar 10 000 para el proyecto

import json
import requests
import feedparser
import pymysql
from sqlalchemy import create_engine
from pandas.io.json import json_normalize
import random
from plotly import plotly
import cufflinks as cf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly import plotly
import cufflinks as cf



#Pagina de api
#https://datos.cdmx.gob.mx/explore/dataset/carpetas-de-investigacion-pgj-cdmx/api/?rows=5000&sort=&refine.ao_hechos=2018    
url = 'https://datos.cdmx.gob.mx/api/records/1.0/search/?dataset=carpetas-de-investigacion-pgj-cdmx&rows=10000&start=0&facet=ao_hechos&facet=mes_hechos&facet=delito&facet=categoria_delito&facet=fiscalia&facet=colonia_hechos&facet=alcaldia_hechos'

response = requests.get(url)
response #Nos aseguramos que la respuesta sea un Ok 200!!!!!

In [ ]:
response = response.json()
response

In [ ]:
response1 = json_normalize(response['records'])

In [ ]:
response1.to_csv('CIPGJ.csv', index=False)

In [ ]:
response1 = pd.read_csv('CIPGJ.csv')
CIPGJ = pd.DataFrame(response1)
CIPGJ.head(2)

In [ ]:
CIPGJ.columns

In [ ]:
CIPGJ = CIPGJ.rename(columns={'fields.agencia':'agencia', 'fields.alcaldia_hechos':'alcaldia_hechos',
       'fields.ao_hechos':'ano_hechos', 'fields.ao_inicio':'ano_inicio', 'fields.calle_hechos':'calle_hechos',
       'fields.calle_hechos2':'Esquina', 'fields.categoria_delito':'categoria_delito',
       'fields.colonia_hechos':'colonia_hechos', 'fields.delito':'delito', 'fields.fecha_hechos':'fecha_hechos',
       'fields.fecha_inicio':'fecha_inicio', 'fields.fiscalia':'fiscalia', 'fields.mes_hechos':'mes_hechos',
       'fields.mes_inicio':'mes_inicio', 'fields.unidad_investigacion':'unidad_investigacion',
       })
CIPGJ.head(2)

In [ ]:
column_order1 = ['datasetid','delito','categoria_delito','fecha_hechos','fecha_inicio','alcaldia_hechos','agencia','fiscalia','unidad_investigacion','colonia_hechos','calle_hechos','Esquina','mes_hechos','fields.geopoint','fields.latitud','fields.longitud','geometry.coordinates','geometry.type']
users_columns = CIPGJ[column_order1]
users_columns.head(2)

In [ ]:
users_columns.dtypes

In [ ]:
bool_series = pd.isnull(users_columns["fields.geopoint"])
nulos=users_columns[bool_series]
nulos

In [ ]:
null_cols = users_columns.isnull().sum()
null_cols

In [ ]:
users_columns[['alcaldia_hechos','unidad_investigacion','colonia_hechos','calle_hechos','Esquina']] = users_columns[['alcaldia_hechos','unidad_investigacion','colonia_hechos','calle_hechos','Esquina']].fillna('')

#Filas donde Hay valores nulos en fields.geopoint
bool_series = pd.isnull(users_columns["fields.geopoint"])
nulos=users_columns[bool_series]
nulos.index

In [ ]:
#Se eliminaron filas con valores nulos
users_columns=users_columns.drop(nulos.index,axis=0)
users_columns

In [ ]:
#se eliminaron fechas de hechos ya que a pesar de ser pocos no tenian otra fecha para poder rellenar el dato
bool_series = pd.isnull(users_columns["fecha_hechos"])
nulos=users_columns[bool_series]
users_columns=users_columns.drop(nulos.index,axis=0)

In [ ]:
null_cols = users_columns.isnull().sum()
null_cols

In [ ]:
users_columns.to_csv('CIPGJ_ordenado_limpio.csv', index=False)

In [ ]:
###########Graficas

In [ ]:
###########Graficas

In [ ]:
users_columns = pd.read_csv('CIPGJ_ordenado_limpio.csv')
users_columns

In [ ]:
users_columns.sort_values(by='mes_hechos', ascending=True)

In [ ]:
mes=users_columns.groupby('mes_hechos').size().sort_values(ascending=False).head(10)

print(dict(mes))

In [ ]:
users_columns.groupby('fiscalia').size().sort_values(ascending=True).head(10)

In [ ]:
data = users_columns.groupby('delito').size()#.sort_values(ascending=False)
data1=dict(data)
data1

In [ ]:
Delitosxmes = None
for m, v in dict(mes).items():
    Delitos = users_columns[(users_columns['mes_hechos']==m)]
    Delitosxmes = pd.concat([Delitosxmes, pd.DataFrame(Delitos.groupby('delito').size())[0]], sort=False, axis=1)
    Delitosxmes = Delitosxmes.rename(columns={0:m})
Delitosxmes.head()

In [ ]:
null_cols = Delitosxmes.isnull().sum()
null_cols

In [ ]:
Delitosxmes.index

In [ ]:
nulos = Delitosxmes.isnull().sum()
Delitosxmes[Delitosxmes.columns] = Delitosxmes[Delitosxmes.columns].fillna(0)
Delitosxmes = Delitosxmes[['Enero','Febrero','Marzo','Abril','Mayo','Agosto','Septiembre','Octubre','Noviembre','Diciembre']]
Delitosxmes.head()
Delitosxmes.to_csv('Delitos2018_CIPGJ.csv')

In [ ]:
Delitosxmes.loc[['ABANDONO DE PERSONA'], :]
incidencia = pd.DataFrame.transpose(Delitosxmes.loc[['ABANDONO DE PERSONA'], :])
incidencia2 = pd.DataFrame.transpose(Delitosxmes)

dict_columns = {k+1:v for k, v in enumerate(incidencia2.columns)}
print('ID Delito', '\t', 'Delito\n')
for k,v in dict_columns.items():
    print(k, '\t\t', v)

In [ ]:
incidencia2 = pd.DataFrame.transpose(Delitosxmes)

deli=[]
i=1
while 0<i and i<177:
    try:
        print(f'Ingresa el ID Delito \ndebe ser un numero entre 1 y {len(dict_columns)} \nque deses agregar a la grafica 2018')
        i=int(input('O un numero negativo para ver la grafica\n'))
    except:
        print("ingresa un numero entre 1 y ", len(dict_columns))
        i=1
        continue
    if 0<i and i<177:
        deli.append(dict_columns[i])

#Las gráficas apreceran automáticamente incrustadas en el notebook
%matplotlib inline

# Definir los datos eje x, eje y
#incidencia = pd.DataFrame.transpose(Delitosxmes.loc[['ABANDONO DE PERSONA'], :])
incidencia = incidencia2[deli]

horas = list(range(len(incidencia)))

# gráfico
# Configurar las caracteristicas del grafico
i=[-1]
def suma():
    i[0]=i[0]+1
    return i[0]

plt.figure(figsize=(20,9))
for h in range(len(deli)):    
    plt.plot(horas, incidencia[deli[h]], label = deli[h], linewidth = 4, color = (random.random(), random.random(), random.random()))
plt.axhline(y=np.mean(incidencia.mean()), linewidth=1, color='r')
#Definir nombres de ejes y titulo
plt.xlabel('Ene                             Feb                             Mar                             Abl                             '
           'May                             Ags                                 Sep                                Oct                             '
           'Nov                             Dic')
plt.ylabel('No de delitos')
plt.title(f'comparatido de {len(deli)} delitos en 2018')
#Mostrar leyenda, cuadricula y figura
plt.legend()
plt.minorticks_on()
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.grid(b=True, which='minor', color='black', linestyle='-', alpha=0.2)
plt.show()



In [ ]:
users_columns.head(1)

In [ ]:
cf.go_offline() #todos los graficos sean de forma local en el cpu

In [ ]:
incidencia2.head()

In [ ]:
incidencia2.iplot(kind='scatter')

In [ ]:
incidencia2.iplot(kind='bar', yTitle='Numero de delitos por mes', title='Incidencia delictiva',
             filename='cufflinks/categorical-bar-chart')

In [ ]:
incidencia2.iplot(kind='bar', barmode='stack', filename='cufflinks/grouped-bar-chart')

In [ ]:
incidencia2.iplot(kind='barh',barmode='stack', bargap=.1, filename='cufflinks/barh')

In [ ]:
delitos=users_columns
delitos

In [ ]:
table = pd.pivot_table(users_columns, 
                       index=['categoria_delito'], 
                       
                       values = 'delito',
                       aggfunc={'delito':'count'}
                      )
table=table.sort_values(by=['delito'], ascending=False)
table.iplot(kind='barh',barmode='stack', bargap=.1, xTitle='Numero de delitos en 2018', title='Incidencia delictiva',
            filename='cufflinks/barh')


In [ ]:
delitos.iplot(kind='scatter', mode='markers', x='mes_hechos', y='delito', filename='cufflinks/simple-scatter')

In [ ]:
#Genere un mapa en google maps donde se muestran los 10000 delitos en el 2018 en df 
#TIENES QUE ACERCAR EL MAPA CON LAS HERRAMIENTAS DE GOOGLE PARA VER CADA PUNTO CORRESPONDIENTE A UN DELITO
#DA CLICK SOBRE EL PUNTO PARA VER SU INFO
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
navegador2 = webdriver.Chrome()
navegador2.get('https://drive.google.com/open?id=1mmf3oPkCRsK2wnOeQqouMSrHK7BGfRVB&usp=sharing')


In [5]:
%matplotlib inline
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"), ),
    center=(52.204793, 360.121558),
    zoom=4
)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [6]:
%matplotlib inline

from ipyleaflet import Marker, Icon, Map

center = (52.204793, 360.121558)

m = Map(center=center, zoom=10)
icon = Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-green.png', icon_size=[38, 95], icon_anchor=[22,94])
mark = Marker(location=center, icon=icon, rotation_angle=90, rotation_origin='22px 94px')
m.add_layer(mark);

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …